In [1]:
# 먼저 GPU 서버를 켜놓는다. GPU 서버의 Embedding 모델을 활용해야 하기 때문.
GPU_URL = "http://localhost:8109/v1" #GPU 서버 URL 및 포트. 끝에 v1이 필요한 이유는 OpenAI랑 맞추기 위함. 형식: some_url:8109/v1
DB_URL = "postgres:j11b109_db_password@3.36.85.129:3346/here_law" #DB URL. 형식: 계정이름:비밀번호@도메인:포트/사용할 DB 이름
COLLECTION_NAME = "case_vector" #지금 넣을 데이터가 들어갈 vector collection

In [2]:
CHUNK_SIZE = 128
ID_PREFIX = "case_"

In [3]:
import os
import sys
os.getcwd()

sys.path.append(os.path.dirname(os.getcwd()))

In [4]:
from backend.fastapi_ec2.agents.tools.embedding_proxy import EmbeddingProxy
from langchain_postgres import PGVector

embedder = EmbeddingProxy(url=GPU_URL)

connection = "postgresql+psycopg://" + DB_URL

vector_store = PGVector(
    embeddings=embedder,
    collection_name=COLLECTION_NAME,
    connection=connection,
    use_jsonb=True,
)

In [5]:
path_list = [
    "/home/ldg/workspace/s2_2/S11P21B109/DB/temp/out1.jsonl",
    "/home/ldg/workspace/s2_2/S11P21B109/DB/temp/out2.jsonl"
]

In [6]:
import pandas as pd
import json

def parse_jsonl(pathlist):
    data_list = []
    for path in pathlist:
        with open(path, "r") as f:
            for line in f:
                data_list.append( json.loads(line) )
    return data_list

In [7]:
alldata = parse_jsonl(path_list)

In [8]:
from langchain_core.documents import Document
docudata = [Document(
    page_content=row["response"]["body"]["choices"][0]["message"]["content"],
    metadata={
        "id" : row["custom_id"]
    }
) for row in alldata] 

In [10]:
from itertools import islice

# Function to split a list into chunks of arbitrary size
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]


In [16]:
results = []
for chunk in chunk_list(docudata, CHUNK_SIZE):
    ids = [ID_PREFIX + doc.metadata["id"] for doc in chunk]  # Extract the list of IDs from the chunk

    existing_ids =  [ID_PREFIX + doc.metadata["id"] for doc in vector_store.get_by_ids(ids)]  # Fetch the existing IDs from the database
    valid_ids = [id_ for id_ in ids if id_ not in existing_ids]
    if len(valid_ids) == 0:
        continue
    to_add = [doc for doc in chunk if ID_PREFIX + doc.metadata["id"] in valid_ids]

    vector_store.add_documents(to_add, ids=[ID_PREFIX + doc.metadata["id"] for doc in to_add])


['case_85832', 'case_85903', 'case_215265', 'case_188582', 'case_215365', 'case_215227', 'case_71778', 'case_86046', 'case_85830', 'case_85834', 'case_85835', 'case_85836', 'case_85831', 'case_85847', 'case_85833', 'case_232167', 'case_85867', 'case_85839', 'case_215267', 'case_85837', 'case_85838', 'case_223851', 'case_215209', 'case_85842', 'case_85840', 'case_215317', 'case_215293', 'case_85848', 'case_71773', 'case_85849', 'case_85850', 'case_133662', 'case_85904', 'case_85857', 'case_85866', 'case_85900', 'case_85898', 'case_85899', 'case_85901', 'case_85865', 'case_85905', 'case_85843', 'case_133663', 'case_85871', 'case_85844', 'case_215219', 'case_188581', 'case_215315', 'case_219997', 'case_85851', 'case_85852', 'case_215369', 'case_85854', 'case_85855', 'case_85853', 'case_232305', 'case_85856', 'case_86038', 'case_137763', 'case_85864', 'case_215281', 'case_85868', 'case_71774', 'case_85913', 'case_232309', 'case_86045', 'case_85872', 'case_133664', 'case_85883', 'case_85863